In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import StandardScaler
import librosa
from google.colab import drive
import os
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2
from sklearn.utils import shuffle
import random
from skimage.transform import resize
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
# Mount Google Drive for access
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# Load centroid data for training from JSON
with open('/content/drive/MyDrive/My-207/X_train_centroid.json', 'r') as file:
    X_train_centroid = np.array(json.load(file))

# Load centroid data for validation from JSON
with open('/content/drive/MyDrive/My-207/X_val_centroid.json', 'r') as file:
    X_val_centroid = np.array(json.load(file))

# Load MFCC data for training from JSON
with open('/content/drive/MyDrive/My-207/X_train_mfcc.json', 'r') as file:
    X_train_mfcc = np.array(json.load(file))

# Load MFCC data for validation from JSON
with open('/content/drive/MyDrive/My-207/X_val_mfcc.json', 'r') as file:
    X_val_mfcc = np.array(json.load(file))

# Load labels for centroid training
with open('/content/drive/MyDrive/My-207/y_train_centroid.json', 'r') as file:
    y_train_list = json.load(file)
y_train = np.array(y_train_list)

# Load labels for centroid validation
with open('/content/drive/MyDrive/My-207/y_val_centroid.json', 'r') as file:
    y_val_list = json.load(file)
y_val = np.array(y_val_list)

# Load labels for MFCC training
with open('/content/drive/MyDrive/My-207/y_train_mfcc.json', 'r') as file:
    y_train_mfcc_list = json.load(file)
y_train_mfcc = np.array(y_train_mfcc_list)

# Load labels for MFCC validation
with open('/content/drive/MyDrive/My-207/y_val_mfcc.json', 'r') as file:
    y_val_mfcc_list = json.load(file)
y_val_mfcc = np.array(y_val_mfcc_list)

In [3]:
# Downsample the MFCC data to match the size of the centroid data
random_indices = random.sample(range(X_train_mfcc.shape[0]), X_train_centroid.shape[0])
X_train_mfcc_downsampled = X_train_mfcc[random_indices]

random_indices = random.sample(range(X_val_mfcc.shape[0]), X_val_centroid.shape[0])
X_val_mfcc_downsampled = X_val_mfcc[random_indices]

# Data normalization
scaler = StandardScaler()

# Flatten and normalize centroid data
X_train_centroid_flat = X_train_centroid.reshape(X_train_centroid.shape[0], -1)
X_val_centroid_flat = X_val_centroid.reshape(X_val_centroid.shape[0], -1)
X_train_centroid_normalized = scaler.fit_transform(X_train_centroid_flat)
X_val_centroid_normalized = scaler.transform(X_val_centroid_flat)
X_train_centroid_normalized = X_train_centroid_normalized.reshape(X_train_centroid.shape)
X_val_centroid_normalized = X_val_centroid_normalized.reshape(X_val_centroid.shape)

# Normalize MFCC data
X_train_mfcc_flat = X_train_mfcc_downsampled.reshape(X_train_mfcc_downsampled.shape[0], -1)
X_val_mfcc_flat = X_val_mfcc_downsampled.reshape(X_val_mfcc_downsampled.shape[0], -1)
X_train_mfcc_normalized = scaler.fit_transform(X_train_mfcc_flat)
X_val_mfcc_normalized = scaler.transform(X_val_mfcc_flat)
X_train_mfcc_normalized = X_train_mfcc_normalized.reshape(X_train_mfcc_downsampled.shape)
X_val_mfcc_normalized = X_val_mfcc_normalized.reshape(X_val_mfcc_downsampled.shape)

# Pad the MFCC data to match the time steps of the centroid data
max_time_steps = max(X_train_centroid_normalized.shape[1], X_val_centroid_normalized.shape[1])
X_train_mfcc_padded = pad_sequences(X_train_mfcc_normalized, maxlen=max_time_steps, dtype='float32', padding='post', truncating='post')
X_val_mfcc_padded = pad_sequences(X_val_mfcc_normalized, maxlen=max_time_steps, dtype='float32', padding='post', truncating='post')


In [4]:
print(X_val_centroid_normalized.shape)
print(X_val_mfcc_padded.shape)


(1785, 251)
(1785, 251, 157)


In [5]:
# Add a new axis to the centroid data
X_train_centroid_normalized = X_train_centroid_normalized[..., np.newaxis]
X_val_centroid_normalized = X_val_centroid_normalized[..., np.newaxis]


In [6]:
# concatenate the centroid and downsampled MFCC data
X_train_combined = np.concatenate((X_train_centroid_normalized, X_train_mfcc_padded), axis=-1)
X_val_combined = np.concatenate((X_val_centroid_normalized, X_val_mfcc_padded), axis=-1)




In [7]:
# Shuffle the training data
X_train_combined, y_train = shuffle(X_train_combined, y_train, random_state=123)

# Shuffle the validation data
X_val_combined, y_val = shuffle(X_val_combined, y_val, random_state=123)


# Encode the labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

# Save the label encoder for later use (optional)
import joblib
label_encoder_path = "label_encoder.pkl"
joblib.dump(label_encoder, label_encoder_path)




['label_encoder.pkl']

In [8]:
# # Build CNN model topology for combined data


# Build CNN model topology for combined data
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(32, kernel_size=5, strides=1, activation='relu', input_shape=(X_train_combined.shape[1], X_train_combined.shape[2])),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(64, kernel_size=5, strides=1, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),  # 1024 units in the Dense layer with ReLU activation
    tf.keras.layers.Dropout(0.5),  # Dropout layer with 50% dropout rate
    tf.keras.layers.Dense(1024, activation='relu'),  # 1024 units in the second Dense layer with ReLU activation
    tf.keras.layers.Dropout(0.5),  # Dropout layer with 50% dropout rate
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

In [9]:
# Create the Adam optimizer with your desired learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.002)
# Compile model
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 247, 32)           25312     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 123, 32)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 119, 64)           10304     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 59, 64)           0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 3776)              0         
                                                                 
 dense (Dense)               (None, 1024)              3

In [12]:
# Train the model with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

history = model.fit(X_train_combined[..., np.newaxis], y_train_encoded,
                    validation_data=(X_val_combined[..., np.newaxis], y_val_encoded),
                    batch_size=32, epochs=100, callbacks=[early_stopping])

Epoch 1/100
143/143 [==============================] - 38s 265ms/step - loss: 0.3776 - accuracy: 0.8588 - val_loss: 1.0137 - val_accuracy: 0.6616
Epoch 2/100
143/143 [==============================] - 27s 187ms/step - loss: 0.3529 - accuracy: 0.8696 - val_loss: 0.9994 - val_accuracy: 0.6891
Epoch 3/100
143/143 [==============================] - 25s 175ms/step - loss: 0.3308 - accuracy: 0.8782 - val_loss: 1.0191 - val_accuracy: 0.6734
Epoch 4/100
143/143 [==============================] - 25s 176ms/step - loss: 0.3128 - accuracy: 0.8863 - val_loss: 1.1844 - val_accuracy: 0.6768
Epoch 5/100
143/143 [==============================] - 26s 183ms/step - loss: 0.2929 - accuracy: 0.8922 - val_loss: 1.1233 - val_accuracy: 0.6818
Epoch 6/100
143/143 [==============================] - 29s 199ms/step - loss: 0.2516 - accuracy: 0.9085 - val_loss: 1.2090 - val_accuracy: 0.6812
Epoch 7/100
143/143 [==============================] - 28s 193ms/step - loss: 0.2474 - accuracy: 0.9153 - val_loss: 1.2318 -

In [ ]:
# Plot the training and validation loss
plt.figure(figsize=(6, 4))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# Plot the training and validation accuracy
plt.figure(figsize=(6, 4))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()